In [1]:
%pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 코드 13-11 필요한 라이브러리 호출
import datetime
import os
from tensorboardX import SummaryWriter

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt

import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 코드 13-12 데이터셋을 내려받은 후 텐서 변환
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(
    root="../chap13/data", train=True, transform=transform, download=True
)

test_dataset = datasets.MNIST(
    root="../chap13/data", train=False, transform=transform, download=True
)

train_loader = DataLoader(
    train_dataset, batch_size=100, shuffle=True, num_workers=4, pin_memory=False
)

test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

In [4]:
# 코드 13-13 인코더 네트워크 생성
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.input1 = nn.Linear(input_dim, hidden_dim)
        self.input2 = nn.Linear(hidden_dim, hidden_dim)
        self.mean = nn.Linear(hidden_dim, latent_dim)
        self.var = nn.Linear(hidden_dim, latent_dim)

        self.LeakyReLU = nn.LeakyReLU(0.2)
        self.training = True

    def forward(self, x):
        h_ = self.LeakyReLU(self.input1(x))
        h_ = self.LeakyReLU(self.input2(h_))
        mean = self.mean(h_)
        log_var = self.var(h_)
        return mean, log_var  # 인코더 네트워크에서 평균과 분산을 반환

In [5]:
# 코드 13-14 디코더 네트워크
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.hidden1 = nn.Linear(latent_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.LeakyReLU = nn.LeakyReLU(0.2)

    def forward(self, x):
        h = self.LeakyReLU(self.hidden1(x))
        h = self.LeakyReLU(self.hidden2(h))
        x_hat = torch.sigmoid(self.output(h))
        return x_hat  # 디코더 결과는 시그모이드를 통과했으므로 0~1 값을 갖습니다.

In [6]:
# 코드 13-15 변형 오토인코더 네트워크
class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder

    def reparameterization(self, mean, var):  # ①
        epsilon = torch.randn_like(var).to(device)
        z = mean + var * epsilon  # z 값 구하기
        return z

    def forward(self, x):
        mean, log_var = self.Encoder(x)  # ②
        z = self.reparameterization(mean, torch.exp(0.5 * log_var))
        x_hat = self.Decoder(z)
        return (
            x_hat,
            mean,
            log_var,
        )  # 디코더 결과와 평균, 표준편차(log를 취한 표준편차)를 반환

In [7]:
# 코드 13-16 인코더와 디코더 객체 초기화
x_dim = 784
hidden_dim = 400
latent_dim = 200
epochs = 30
batch_size = 100

encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim=hidden_dim, output_dim=x_dim)

model = Model(Encoder=encoder, Decoder=decoder).to(device)

In [8]:
# 코드 13-17 손실 함수 정의
def loss_function(x, x_hat, mean, log_var):  # ①
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction="sum")
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    return reproduction_loss, KLD


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [9]:
# 코드 13-18 모델 학습 함수 정의
saved_loc = "scalar/"  # 텐서보드에서 사용할 경로
writer = SummaryWriter(saved_loc)  # ①

model.train()


def train(epoch, model, train_loader, optimizer):
    train_loss = 0
    for batch_idx, (x, _) in enumerate(train_loader):
        x = x.view(batch_size, x_dim)
        x = x.to(device)

        optimizer.zero_grad()
        x_hat, mean, log_var = model(x)
        BCE, KLD = loss_function(x, x_hat, mean, log_var)
        loss = BCE + KLD
        writer.add_scalar(
            "Train/Reconstruction Error",
            BCE.item(),
            batch_idx + epoch * (len(train_loader.dataset) / batch_size),
        )  # ②
        writer.add_scalar(
            "Train/KL-Divergence",
            KLD.item(),
            batch_idx + epoch * (len(train_loader.dataset) / batch_size),
        )
        writer.add_scalar(
            "Train/Total Loss",
            loss.item(),
            batch_idx + epoch * (len(train_loader.dataset) / batch_size),
        )

        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}".format(
                    epoch,
                    batch_idx * len(x),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item() / len(x),
                )
            )

    print(
        "======> Epoch: {} Average loss: {:.4f}".format(
            epoch, train_loss / len(train_loader.dataset)
        )
    )

In [10]:
# 코드 13-19 모델 평가 함수 정의
def test(epoch, model, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (x, _) in enumerate(test_loader):
            x = x.view(batch_size, x_dim)
            x = x.to(device)
            x_hat, mean, log_var = model(x)
            BCE, KLD = loss_function(x, x_hat, mean, log_var)
            loss = BCE + KLD

            writer.add_scalar(
                "Test/Reconstruction Error",
                BCE.item(),
                batch_idx + epoch * (len(test_loader.dataset) / batch_size),
            )  # 테스트 데이터셋에 대해서도 오차를 로그에 저장
            writer.add_scalar(
                "Test/KL-Divergence",
                KLD.item(),
                batch_idx + epoch * (len(test_loader.dataset) / batch_size),
            )
            writer.add_scalar(
                "Test/Total Loss",
                loss.item(),
                batch_idx + epoch * (len(test_loader.dataset) / batch_size),
            )
            test_loss += loss.item()

            if batch_idx == 0:
                n = min(x.size(0), 8)
                comparison = torch.cat([x[:n], x_hat.view(batch_size, x_dim)[:n]])
                grid = torchvision.utils.make_grid(comparison.cpu())
                writer.add_image(
                    "Test image - Above: Real data, below: reconstruction data",
                    grid,
                    epoch,
                )

In [11]:
# 코드 13-20 모델 학습
from tqdm.auto import tqdm

for epoch in tqdm(range(0, epochs)):
    train(epoch, model, train_loader, optimizer)
    test(epoch, model, test_loader)
    print("\n")
writer.close()  # ①

  0%|          | 0/30 [00:00<?, ?it/s]

Train Epoch: 0 [0/60000 (0%)]	 Loss: 545.295156
Train Epoch: 0 [10000/60000 (17%)]	 Loss: 194.420605
Train Epoch: 0 [20000/60000 (33%)]	 Loss: 187.118809
Train Epoch: 0 [30000/60000 (50%)]	 Loss: 160.012686
Train Epoch: 0 [40000/60000 (67%)]	 Loss: 161.440273
Train Epoch: 0 [50000/60000 (83%)]	 Loss: 149.570811
======> Epoch: 0 Average loss: 173.6872


Train Epoch: 1 [0/60000 (0%)]	 Loss: 139.700586
Train Epoch: 1 [10000/60000 (17%)]	 Loss: 139.730322
Train Epoch: 1 [20000/60000 (33%)]	 Loss: 124.656973
Train Epoch: 1 [30000/60000 (50%)]	 Loss: 123.389258
Train Epoch: 1 [40000/60000 (67%)]	 Loss: 122.993828
Train Epoch: 1 [50000/60000 (83%)]	 Loss: 124.900615
======> Epoch: 1 Average loss: 128.0428


Train Epoch: 2 [0/60000 (0%)]	 Loss: 121.788740
Train Epoch: 2 [10000/60000 (17%)]	 Loss: 118.400615
Train Epoch: 2 [20000/60000 (33%)]	 Loss: 116.927939
Train Epoch: 2 [30000/60000 (50%)]	 Loss: 117.731973
Train Epoch: 2 [40000/60000 (67%)]	 Loss: 118.080342
Train Epoch: 2 [50000/60000 (8

In [16]:
# 코드 13-21 텐서보드에서 오차 확인
%load_ext tensorboard
%tensorboard --logdir scalar --port=6013

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6013 (pid 78976), started 0:00:05 ago. (Use '!kill 78976' to kill it.)